In [1]:
import geopandas as gpd
gpd.options.use_pygeos=False

/home/lucas/miniconda3/envs/sips/lib/python3.7/site-packages/geopandas/_compat.py:78: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
import os, sys, io
from shapely import geometry

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from area import area
import requests
from functools import partial
from shapely.ops import transform,linemerge, unary_union, polygonize
from shapely.affinity import affine_transform
from functools import partial
import pyproj

from PIL import Image
import descarteslabs as dl

In [4]:
root = os.path.abspath(os.path.join(os.getcwd(),'..','..','..'))

In [5]:
sys.path.append(root)

In [6]:
from solarpv.utils import V_inv

In [7]:
API_KEY = '<YOUR-API-KEY>'

In [8]:
def img_poly(ft_poly,mmpix):

    #V_inv -> lat,lon
    #for each point, go centroid->pt
    #print(centroid)
    img_coords = []
    for pt in list(ft_poly.exterior.coords):
        #pt->lon,lat
        #centroid->lon,lat
        dist, angle, dummy = V_inv((ft_poly.centroid.y,ft_poly.centroid.x),(pt[1],pt[0]))
        dist=dist*1000
        #print ((400+2*(dist/mmpix*np.cos(2.*np.pi*(angle)/360.)),400+2*(dist/mmpix*np.sin(2*np.pi*angle/360.))))
        img_coords.append((400+2*(dist/mmpix*np.cos(2.*np.pi*(angle-90.)/360.)),400+2*(dist/mmpix*np.sin(2*np.pi*(angle-90.)/360.))))
    #V_inv(point1, point2
    #v_dir
    return img_coords

In [9]:
def lonlat2pixXY(pt,dt):
    lon = pt[0]
    lat = pt[1]
    Y = (lat-dt[3]-dt[4]/dt[1]*(lon-dt[0]))/(dt[5]-dt[2]*dt[4]/dt[1])
    #print Y
    X = (lon-dt[0]-Y*dt[2])/dt[1]
    #print (lon,dt[0],X)
    #print X
    return [int(X),int(Y)]

In [10]:
def inspect_sample(row,src = 'google'):
    
    ft_shp = row['geometry']
    if ft_shp.type=='MultiPolygon':
        run_polys = list(ft_shp)
    else:
        run_polys = [ft_shp]
        
    for ii_p,ft_poly in enumerate(run_polys):
        centroid = ft_poly.centroid ##lon,lat
        print ('centroid',centroid)
        ft_bbox = ft_poly.bounds
        
        Dx = ft_poly.bounds[2]-ft_poly.bounds[0]
        Dy = ft_poly.bounds[3]-ft_poly.bounds[1]
        print (max(abs(Dx),abs(Dy)))
        ft_box = geometry.box(*ft_poly.buffer(max(abs(Dx),abs(Dy))).bounds)
        
        
        fig, axs=plt.subplots(1,2,figsize=(24,12))

        if src=='google':
            box_sides = (V_inv((ft_bbox[1],ft_bbox[0]),(ft_bbox[1],ft_bbox[2]))[0]*1000,
                             V_inv((ft_bbox[1],ft_bbox[0]),(ft_bbox[3],ft_bbox[0]))[0]*1000)
            #print (box_sides)
            side_len = np.ceil(max(box_sides))
            print ('side_len (m)',side_len)

            zoom_dict = dict(zip(range(1,21),[156543.03392 * np.cos(centroid.xy[1][0] * np.pi / 180) / np.power(2, z) for z in range(1,21)]))
            #print (zoom_dict) ##<-- METERS PER PIX, not side length

            zoom = np.max(np.argwhere(np.array([(zoom_dict[k]*400-max(box_sides)) for k in range(1,21)])>0.))+1


            #min(zoom_dict.keys(), key=(lambda k: (zoom_dict[k]-max(box_sides))))
            print ('zoom',zoom, zoom_dict[zoom], 'area',area(geometry.mapping(row['geometry'])), np.array(centroid))
            pix_poly = img_poly(ft_poly,zoom_dict[zoom])

            urlstr = ''.join(["""https://maps.googleapis.com/maps/api/staticmap?center=""",
                        str(centroid.xy[1][0])+""","""+str(centroid.xy[0][0]),
                        """&zoom="""+str(zoom),
                        """&size=400x400&scale=2&maptype=satellite&format=png&key=""", str(API_KEY)])

            r = requests.get(urlstr, allow_redirects=True)
            #print (r.content)

            image_data = r.content
            image = Image.open(io.BytesIO(image_data))
            image = image.convert('RGB')

            arr = np.asarray(image)
            
            axs[0].imshow(arr)
            xs, ys = geometry.Polygon(pix_poly).exterior.xy
            #print (geometry.Polygon(pix_poly))
            #print (xs,ys)
            axs[0].plot(xs,ys, color='c', linewidth=2.)
            axs[0].set_title('Google Basemap (indeterminate date), {:.2f}m/px'.format(zoom_dict[zoom]), fontsize=20)
        
            
        elif src=='SPOT':
            scenes, ctx = dl.scenes.search(
            ft_box,
            products=['airbus:oneatlas:spot:v2'],
            start_datetime='2015-09-01',  end_datetime='2018-12-31', cloud_fraction=0.2, limit=10, processing_level='surface'
            )
            scenes = sorted(scenes, key=lambda k: k.properties.date, reverse=True)
            print ([s.properties.date for s in scenes])
            
            arr_SPOT = scenes[0].ndarray("red green blue", ctx)

            for s in scenes[1:]:
                fill_por = np.sum(arr_SPOT>0)/np.prod([*arr_SPOT.shape])
                #print (fill_por, np.sum(arr_S2), np.prod([*arr_S2.shape]))
                new_arr = s.ndarray("red green blue", ctx)
                #print ('shapes',new_arr.shape, np.sum(new_arr>0))

                arr_SPOT[new_arr.mask==False]=new_arr.data[new_arr.mask==False]


            scene_crs = scenes[0].properties['crs']
            dt = scenes[0].properties['geotrans']
            dt_shapely = [dt[1],dt[2],dt[4],dt[5],dt[0],dt[3]]

            WGS84 = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"

            wgs_proj = pyproj.Proj(WGS84)
            utm_proj = pyproj.Proj("+init="+scene_crs, preserve_units=True)


            dt[0] = utm_proj(*ctx.bounds[0:2])[0]
            dt[3] = utm_proj(*ctx.bounds[2:])[1]

            projection_func = partial(pyproj.transform, wgs_proj, utm_proj)

            utm_poly = transform(projection_func, ft_poly)
            pix_poly = geometry.Polygon([lonlat2pixXY(c,dt) for c in list(utm_poly.exterior.coords)])

            axs[0].imshow((np.swapaxes(np.swapaxes(arr_SPOT.data,0,2),0,1)/256).clip(0.,1.))
            xs,ys = pix_poly.exterior.xy
            axs[0].plot(xs,ys,c='c',linewidth=2)
            
            axs[0].set_title('SPOT {}, 10m/px, , {:d}x{:d}px'.format(scenes[0].properties.date,arr_SPOT.shape[1],arr_SPOT.shape[2]), fontsize=20)
        
            
        
        
        
        
        scenes, ctx = dl.scenes.search(
            ft_box,
            products=['sentinel-2:L1C'],
            start_datetime='2018-09-01',  end_datetime='2018-12-31', cloud_fraction=0.2, limit=10,
            )
        
        if len(scenes)>0:
            scenes = sorted(scenes, key=lambda k: k.properties.cloud_fraction, reverse=False)

            arr_S2 = scenes[0].ndarray("red green blue", ctx)

            for s in scenes[1:]:
                fill_por = np.sum(arr_S2>0)/np.prod([*arr_S2.shape])
                #print (fill_por, np.sum(arr_S2), np.prod([*arr_S2.shape]))
                new_arr = s.ndarray("red green blue", ctx)
                #print ('shapes',new_arr.shape, np.sum(new_arr>0))

                arr_S2[new_arr.mask==False]=new_arr.data[new_arr.mask==False]


            scene_crs = scenes[0].properties['crs']
            dt = scenes[0].properties['geotrans']
            dt_shapely = [dt[1],dt[2],dt[4],dt[5],dt[0],dt[3]]

            WGS84 = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"

            wgs_proj = pyproj.Proj(WGS84)
            utm_proj = pyproj.Proj("+init="+scene_crs, preserve_units=True)


            dt[0] = utm_proj(*ctx.bounds[0:2])[0]
            dt[3] = utm_proj(*ctx.bounds[2:])[1]

            projection_func = partial(pyproj.transform, wgs_proj, utm_proj)

            utm_poly = transform(projection_func, ft_poly)
            pix_poly = geometry.Polygon([lonlat2pixXY(c,dt) for c in list(utm_poly.exterior.coords)])

            axs[1].imshow((np.swapaxes(np.swapaxes(arr_S2.data,0,2),0,1)/10000*2.5).clip(0.,1.))
            xs,ys = pix_poly.exterior.xy
            axs[1].plot(xs,ys,c='c',linewidth=2)
        axs[1].set_title('Sentinel-2 (Q4 2018), 1.5m/px, , {:d}x{:d}px'.format(arr_S2.shape[1],arr_S2.shape[2]), fontsize=20)
        fig.suptitle(f'uid: {row["unique_id"]},lon:{row["geometry"].representative_point().x:.5f},lat:{row["geometry"].representative_point().y:.5f}', fontsize=26)
        #plt.tight_layout()
        fig.savefig(os.path.join(root,'data','hand_verify',str(row['unique_id'])+'.png'))
        #plt.show()
        plt.close()
        
    
    return True


In [11]:
gdf = gpd.read_file(os.path.join(root,'data','ABCD_finalized.geojson'))

In [12]:
import glob

In [70]:
pos_labels = glob.glob(os.path.join(root,'data','hand_verify','pos','*'))

In [71]:
neg_labels = glob.glob(os.path.join(root,'data','hand_verify','neg','*'))

In [72]:
ind_labels = glob.glob(os.path.join(root,'data','hand_verify','ind','*'))

In [47]:
all_ids = [f.split('/')[-1].split('.')[0] for f in pos_labels+neg_labels+ind_labels]

In [48]:
all_ids

['840',
 '5826',
 '7614',
 '21144',
 '64968',
 '14722',
 '54988',
 '1165',
 '11213',
 '62483',
 '52218',
 '50247',
 '5812',
 '30444',
 '10237',
 '46291',
 '27278',
 '28792',
 '5690',
 '63779',
 '18494',
 '37200',
 '15756',
 '5838',
 '46014',
 '39305',
 '1162',
 '2399',
 '68517',
 '2705',
 '66794',
 '7213',
 '64593',
 '23569',
 '60193',
 '28939',
 '53953',
 '53426',
 '26501',
 '34756',
 '21547',
 '27992',
 '26308',
 '3192',
 '13630',
 '9741',
 '17037',
 '52161',
 '2861',
 '5289',
 '17638',
 '60071',
 '21452',
 '36846',
 '49044',
 '43538',
 '5022',
 '25854',
 '66877',
 '24054',
 '1798',
 '24280',
 '14663',
 '4859',
 '20579',
 '56585',
 '53532',
 '14077',
 '27599',
 '22081',
 '22468',
 '48099',
 '21900',
 '47035',
 '67894',
 '59294',
 '4297',
 '38775',
 '47672',
 '63692',
 '46187',
 '11954',
 '48623',
 '14903',
 '22201',
 '36942',
 '8045',
 '22456',
 '2180',
 '40809',
 '39554',
 '49475',
 '25720',
 '39377',
 '58996',
 '10633',
 '29560',
 '65062',
 '50345',
 '67441',
 '26571',
 '54405',
 '

In [79]:
1e4

10000.0

In [22]:
bins = [1e2,1e3,1e4,1e5,1e6,1e9]

In [49]:
slice_ids = [int(ii) for ii in all_ids]

In [50]:
for ii in range(len(bins)-1):
    print (ii,((gdf['area']>bins[ii]) & (gdf['area']<=bins[ii+1])).sum())

0 2858
1 26328
2 31032
3 7736
4 707


In [51]:
for ii in range(len(bins)-1):
    print (len(gdf.loc[gdf['unique_id'].isin(slice_ids) & (gdf['area']>bins[ii]) &(gdf['area']<=bins[ii+1]),:]))

191
365
481
198
173


In [28]:
ids = {ii:gdf.loc[(gdf['area']>bins[ii]) &(gdf['area']<=bins[ii+1]),'unique_id'].values.tolist() for ii in range(len(bins)-1)}

In [34]:
np.random.choice(range(7),3)

array([0, 0, 3])

In [66]:
ids_0 = np.random.choice(ids[0],0).tolist()
ids_3 = np.random.choice(ids[3],0).tolist()
ids_4 = np.random.choice(ids[4],10).tolist()

In [67]:
#inspect_set = gdf.iloc[np.random.choice(range(len(gdf)),N_inspect),:]
inspect_set = gdf.loc[gdf['unique_id'].isin(ids_0+ids_3+ids_4),:]

In [68]:
for row in inspect_set.iterrows():
    print (row[0])

3362
3781
6526
11315
25358
25642
51349
63192
63570
63662


In [69]:
for row in inspect_set.iloc[0:,].iterrows():
    print (row[0])#['unique_id'])
    inspect_sample(row[1],src = 'google')

3362
centroid POINT (125.3550126865927 45.76778983204898)
0.020259544911780836
side_len (m) 2252.0
zoom 14 6.665003316759743 area 1955813.6269617288 [125.35501269  45.76778983]


/home/lucas/miniconda3/envs/sips/lib/python3.7/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)


3781
centroid POINT (120.0677506411075 34.37052825508912)
0.024433782230907752
side_len (m) 2711.0
zoom 14 7.88642857572294 area 2598340.0400367775 [120.06775064  34.37052826]


/home/lucas/miniconda3/envs/sips/lib/python3.7/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)


6526
centroid POINT (112.5386477034068 30.19541430178121)
0.018935224662723016
side_len (m) 2100.0
zoom 14 8.258209322788131 area 1800324.1166490389 [112.5386477  30.1954143]


...

/home/lucas/miniconda3/envs/sips/lib/python3.7/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)


63570
centroid POINT (119.8988732789843 31.55316186982336)
0.017590089348161086
side_len (m) 1671.0
zoom 14 8.142024453429416 area 1086031.3872476274 [119.89887328  31.55316187]


/home/lucas/miniconda3/envs/sips/lib/python3.7/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)


63662
centroid POINT (119.5155362443963 31.83447065772674)
0.026698468795455454
side_len (m) 2961.0
zoom 14 8.11737839760535 area 2077915.0208343156 [119.51553624  31.83447066]


/home/lucas/miniconda3/envs/sips/lib/python3.7/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)


### Assess final

In [73]:
pos_ids = [int(f.split('/')[-1].split('.')[0]) for f in pos_labels]
neg_ids = [int(f.split('/')[-1].split('.')[0]) for f in neg_labels]

In [74]:
for ii in range(len(bins)-1):
    pos = ((gdf['unique_id'].isin(pos_ids)) & (gdf['area']>bins[ii]) & (gdf['area']<=bins[ii+1])).sum()
    neg = ((gdf['unique_id'].isin(neg_ids)) & (gdf['area']>bins[ii]) & (gdf['area']<=bins[ii+1])).sum()
    print (ii,pos,neg, pos/(pos+neg))

0 197 4 0.9800995024875622
1 360 5 0.9863013698630136
2 478 3 0.9937629937629938
3 196 5 0.9751243781094527
4 194 5 0.9748743718592965


In [78]:
pos = len(gdf[gdf['unique_id'].isin(pos_ids)])
neg = len(gdf[gdf['unique_id'].isin(neg_ids)])
print (pos,neg, pos/(pos+neg))

1425 22 0.9847961299239807


In [81]:
(478+196+194)/(478+196+194+13)

0.985244040862656